In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.set_printoptions(precision=3, linewidth=500, threshold=500, suppress=True, edgeitems=5)

pd.set_option('display.max_rows', 200)
pd.set_option('display.min_rows', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 100)

%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from numpy.random import randn
from numpy import diag, ndarray



In [3]:
import sys
sys.path.append('..')

In [5]:
from utils.kronecker import KroneckerOperator, KroneckerProduct, KroneckerSum, KroneckerDiag, _ProductChain, _SumChain
from utils.linalg import kronecker_product_literal, kronecker_sum_literal, vec, ten


In [87]:
N1 = 6
N2 = 5
N3 = 4
N4 = 3

A1 = randn(N1, N1)
A2 = randn(N2, N2)
A3 = randn(N3, N3)
A4 = randn(N4, N4)


A11 = randn(N1, N1)
A22 = randn(N2, N2)
A33 = randn(N3, N3)
A44 = randn(N4, N4)

X = randn(N4, N3, N2, N1)
D = randn(N4, N3, N2, N1)

In [89]:
np.reduce()

AttributeError: module 'numpy' has no attribute 'reduce'

In [76]:
a = np.array([1, 2, 3, 4])

In [77]:
ma = np.ma.masked_array(a, mask=[0, 0, 1, 0])

In [80]:
type(ma)

numpy.ma.core.MaskedArray

In [82]:
isinstance(ma, ndarray)

True

In [86]:
ma.data

array([1, 2, 3, 4])

In [85]:
~np.isnan([np.nan, 4, 5, 6])

array([False,  True,  True,  True])